# Spatial Filtering
In this notebook, we will go through some basic methods of image filtering in the spatial domain.

There will be a brief description of each method in the notebook, but you are encouraged to research each method yourself and try it on different images.

## Section 0. Preparing the Notebook
We start by importing the necessary libraries and then loading a sample image to work on.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing necessary packages
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from solutions import *
from utils import *

In [ ]:
# loading the sample image and setting the colormap for pyplot
image = np.float64(cv.imread('data/man.bmp', cv.IMREAD_GRAYSCALE) / 255)
plt.set_cmap('Greys_r')
_ = plt.imshow(image), plt.axis('off')

## Section 1. What is a Kernel?
A very basic concept in spatial image filtering is a kernel. A kernel in its simplest form is a matrix which we slide across the different pixels in the image. In each position, the values of the kernel and the image are multiplied together, and the sum of these values will be the output for that position. In mathematics, this operation is called a correlation and is denoted by $ \circledast $. Below is an example of this operation:

$$
\begin{bmatrix}
    1 & 3 & 5 & 6 \\
    2 & 5 & 1 & 8 \\
    2 & 3 & 5 & 9 \\
    1 & 4 & 4 & 6
\end{bmatrix}
\circledast 
\frac{1}{9} \cdot
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & 1 \\
    1 & 1 & 1 
\end{bmatrix}
=
\begin{bmatrix}
3 & 5 \\
3 & 5
\end{bmatrix}
$$

What we did here, was averaging over a $3 \times 3 $ window, which is commonly known as average or mean filtering.

### Section 1.1. Implementing a mean filter
Now, write a piece of code which creates a mean kernel, and applies it to the sample image. Try the mean filter with different kernel sizes and compare the results.

In [ ]:
# Implementing a mean kernel constructor
def meanKernel(kernel_size : Iterable[int]) -> np.ndarray:
    """
    Parameters:
    - kernel_size : Iterable[int, int]
        An Iterable of two integers, which determines the size of the kernel.
    Returns:
    - kernel : np.ndarray
        A mean kernel in np.ndarray format, with the specified size,
        and with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()


# Creating the kernels and applying the filter
kernel_size = (5, 5)
kernel = meanKernel(kernel_size)
kernel_ref = meanKernelRef(kernel_size)
image_filtered = cv.filter2D(image, cv.CV_64F, kernel)
image_filtered_ref = cv.filter2D(image, cv.CV_64F, kernel_ref)

# Showing the results
_ = plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
_ = plt.subplot(1, 3, 1), plt.imshow(image_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your output')
_ = plt.subplot(1, 3, 2), plt.imshow(image_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference output')
_ = plt.subplot(1, 3, 3), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original image')

### Section 1.2. Implementing a gaussian filter
In the previous section, you saw that the mean filter, while removing the fine details, also blurs the important parts of the image, such as the edges of the buildings. A type of kernel which does less damage to edges is the gaussian kernel. The coefficients of a gaussian kernel $ G $ with the center $ c $ can be described by integrating over the familiar gaussian distribution formula:

$ G_{(i,j)} = \large \frac{1}{\sigma \sqrt{2 \pi}} \cdot \exp(-\frac{(i - c_i)^2 + (j - c_j)^2 }{\sigma^2}) $

Here is a sample $ 3 \times 3 $ gaussian kernel with $ \sigma = 1 $.

$
G = 
\begin{bmatrix}
0.075 & 0.124 & 0.075 \\
0.124 & 0.204 & 0.124 \\
0.075 & 0.124 & 0.075
\end{bmatrix}
$

Using openCV, you can get a 1-D gaussian kernel by calling the ``getGaussianKernel`` function. Use this to create a gaussian kernel like the one described above.

In [ ]:
# Implementing a gaussian kernel constructor
def gaussianKernel(kernel_size : Iterable[int], sigma : float) -> np.ndarray:
    """
    Parameters:
    - kernel_size : Iterable[int]
        An Iterable of two integers, which determines the size of the kernel.
    - sigma : float
        The sigma parameter in the gaussian distribution.
    Returns:
    - kernel : np.ndarray
        A gaussian kernel in np.ndarray format, with the specified size,
        and with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()


# Creating the kernels and applying the filter
kernel_size = (5, 5)
sigma = 2
kernel = gaussianKernel(kernel_size, sigma)
kernel_ref = gaussianKernelRef(kernel_size, sigma)
image_filtered = cv.filter2D(image, cv.CV_64F, kernel)
image_filtered_ref = cv.filter2D(image, cv.CV_64F, kernel_ref)

# Showing the results
_ = plt.figure(figsize=(18, 6))
_ = plt.subplot(1, 3, 1), plt.imshow(image_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your output')
_ = plt.subplot(1, 3, 2), plt.imshow(image_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference output')
_ = plt.subplot(1, 3, 3), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original image')

## Section 2. Edge Detection Filters
The mean and gaussian filters above mostly serve to remove details from an image. However, we can use spatial filtering to extract details from an image, e.g. the edges of objects.

### Section 2.1. First-Derivative Filters

#### Section 2.1.1 Basic First-Derivative Filters
A very simple way to detect edges, i.e. the areas where there is a sudden change in pixel intensity levels, is by calculating the derivative of an image along a certain axis. There are several derivative kernels that we can use:


![Derivative Kernels](figures/derivative_kernels.jpg "Derivative Kernels")

Implement some of these kernels below and observe the output edge map.

In [ ]:
# Implementing a derivative kernel constructor
def derivateKernel(direction : str, mode : str) -> np.ndarray:
    """
    Parameters:
    - direction : ['h' | 'v']
        Determines the axis along which derivation takes place. 'h' for horizontal
        and 'v' for vertical differentiation.
    - mode : ['c' | 'f']
        Determines the type of differentiation. 'c' for central difference, and 'f'
        for forward difference.
    Returns:
    - kernel : np.ndarray
        A derivative kernel in np.ndarray format, with the specified 
        characteristics and with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()


# Creating the kernels and applying the filter
direction = 'v'
mode = 'f'
kernel = derivateKernel(direction, mode)
kernel_ref = derivateKernelRef(direction, mode)
edge_map = cv.filter2D(image, cv.CV_64F, kernel)
edge_map_ref = cv.filter2D(image, cv.CV_64F, kernel_ref)

# Showing the results
_ = plt.figure(figsize=(18, 6))
_ = plt.subplot(1, 3, 1), plt.imshow(edge_map, vmin=-1, vmax=1), plt.axis('off'), plt.title('Your Derivative Map')
_ = plt.subplot(1, 3, 2), plt.imshow(edge_map_ref, vmin=-1, vmax=1), plt.axis('off'), plt.title('Reference Derivative Map')
_ = plt.subplot(1, 3, 3), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

As you can see, the edge map has a gray-ish tone, since the minimum intensity for pixels is now $ -1 $ instead of $ 0 $, and therefore smooth areas in the original image look gray in the intensity map. We can use the absolute value of the map to show a pure representation of edges.

In [ ]:
# Calculating the absolute value of the maps
edge_map = np.abs(edge_map)
edge_map_ref = np.abs(edge_map_ref)

# Showing the results
_ = plt.figure(figsize=(18, 6))
_ = plt.subplot(1, 3, 1), plt.imshow(edge_map), plt.axis('off'), plt.title('Magnitude of Your Derivative Map')
_ = plt.subplot(1, 3, 2), plt.imshow(edge_map_ref), plt.axis('off'), plt.title('Magnitude of Reference Derivative Map')
_ = plt.subplot(1, 3, 3), plt.imshow(image), plt.axis('off'), plt.title('Original Image')

However, by doing so we have destroyed the data about the direction of the edges, meaning that we no longer know whether a certain point in the edge map shows a bright-to-dark or a dark-to-bright transition. Also, notice that we have only gathered the edges along a certain direction in our edge map. One way of dealing with this issue is by dividing the edge data into two maps of edge *magnitude* and *orientation*. You might also hear the term *gradient map* used for this representation.

For two derivate functions $ d_x $ and $ d_y $, the magnitude and orientation of the gradient $ G $ can be calculated as follows:

$
{\parallel G \parallel}^2 = d_x^2 + d_y^2 
$

$
\angle G = atan({d_y},{d_x})
$

Using the derivative maps of an image, create a gradient map of the sample image.

In [ ]:
# Implementing a gradient map constructor
def gradientMap(image : np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Parameters:
    - image : np.ndarray
        An image in np.ndarray format, with dtype=np.float64.
        
    Returns
    - gradient_magnitude : np.ndarray
        An np.ndarray representation of gradient magnitudes, with dtype=np.float64.
    - gradient_orientation : np.ndarray
        An np.ndarray representation of gradient orientations. Values can range from
        -pi to +pi. Should also have dtype=np.float64.
    
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()


# Applying the filter
gradient_magnitude, gradient_orientation = gradientMap(image)
gradient_magnitude_ref, gradient_orientation_ref = gradientMapRef(image)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(gradient_magnitude), plt.axis('off'), plt.title('Your Gradient Magnitude')
_ = plt.subplot(2, 2, 2), plt.imshow(gradient_orientation, vmin=-np.pi, vmax=np.pi, cmap='twilight'), plt.axis('off'), plt.title('Your Gradient Orientation')
_ = plt.subplot(2, 2, 3), plt.imshow(gradient_magnitude_ref), plt.axis('off'), plt.title('Reference Gradient Magnitude')
_ = plt.subplot(2, 2, 4), plt.imshow(gradient_orientation_ref, vmin=-np.pi, vmax=np.pi, cmap='twilight'), plt.axis('off'), plt.title('Reference Gradient Orientation')

#### Section 2.1.2. Prewitt and Sobel Kernels
While the derivative kernels shown above can work perfectly well for a noiseless image, they tend to give distorted outputs for noisy images. One way of countering this, is by applying a smoothing filter, i.e. a mean or gaussian blurring filter, in one direction; and then applying a derivative kernel in the other direction. These two consecutive filters can be represented by a single kernel, in the fashion shown below.

![image](figures/prewitt.jpg)

![image](figures/sobel.jpg)

In the case where the smoothing kernel is a 3-radius averaging kernel, we call the resulting kernel a Prewitt kernel, and if a 3-radius gaussian kernel is used, we call it a Sobel kernel.

Implement the constructor functions for these smooth derivation kernels, and see their results on a sample noisy image. Use a forward-difference derivation kernel, so Prewitt and Sobel kernels can be constructed.

In [ ]:
# Creating a noisy image
image_noisy = gaussianNoise(image, 0.1)

# Implementing a smooth derivative kernel constructor
def smoothDerivativeKernel(direction : str, kernel_type : str, radius : int, sigma : float = None) -> np.ndarray:
    """
    Parameters:
    - direction : ['h' | 'v']
        Determines the axis along which derivation takes place. 'h' for horizontal
        and 'v' for vertical differentiation.
    - kernel_type : ['g' | 'm']
        Determines the type of the smoothing kernel. 'g' for gaussian, and 'm' for 
        mean.
    - radius : int
        Determines the radius of the smoothing kernel.
    - sigma : float
        Determines the sigma parameter for gaussian kernels.
    Returns:
    - kernel : np.ndarray
        A kernel in np.ndarray format, with the specified characteristics and with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()


# Creating horizontal Sobel kernels
kernel_h = derivateKernel('h', 'f')
kernel_h_smooth = smoothDerivativeKernel('h', 's', 3, 1)
kernel_h_ref = derivateKernelRef('h', 'f')
kernel_h_smooth_ref = smoothDerivativeKernelRef('h', 'g', 3, 1)

edge_map = np.abs(cv.filter2D(image_noisy, cv.CV_64F, kernel_h))
edge_map_smooth = np.abs(cv.filter2D(image_noisy, cv.CV_64F, kernel_h_smooth))
edge_map_ref = np.abs(cv.filter2D(image_noisy, cv.CV_64F, kernel_h_ref))
edge_map_smooth_ref = np.abs(cv.filter2D(image_noisy, cv.CV_64F, kernel_h_smooth_ref))

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(edge_map, vmin=0, vmax=1), plt.axis('off'), plt.title('Your  Edge Map')
_ = plt.subplot(2, 2, 2), plt.imshow(edge_map_smooth, vmin=0, vmax=1), plt.axis('off'), plt.title('Your  Smoothed Edge Map')
_ = plt.subplot(2, 2, 3), plt.imshow(edge_map_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Edge Map')
_ = plt.subplot(2, 2, 4), plt.imshow(edge_map_smooth_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Smoothed Edge Map')

### Section 2.2. Second-Derivative Kernels

#### Section 2.2.1 Basic Second-Derivative Kernels
Another way of detecting edges is by using a second derivative kernel. The second derivative, in discrete sequences is defined as follows:

$ \Large \frac{\partial I(t)}{\partial t} = \normalsize I(t-1) - 2I(t) + I(t+1)$

In these filters, rather than the changes in pixel intensity, the *convexity* of pixel values is measured, and edges are marked with *zero-crossings*, i.e. parts of the image where the value of the second-derivative passes from negative to positive, or vice versa.

Implement a second derivative kernel and see its effect on the sample image.

In [ ]:
# Implementing a second derivative kernel constructor
def secondDerivateKernel(direction : str) -> np.ndarray:
    """
    Parameters:
    - direction : ['h' | 'v']
        Determines the axis along which derivation takes place. 'h' for horizontal
        and 'v' for vertical differentiation.
    Returns:
    - kernel : np.ndarray
        A second derivative kernel in np.ndarray format, with the specified direction
        and with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()


# Creating the kernels and applying the filter
direction = 'h'
kernel = secondDerivateKernel(direction)
kernel_ref = secondDerivateKernelRef(direction)
edge_map = cv.filter2D(image, cv.CV_64F, kernel)
edge_map_ref = cv.filter2D(image, cv.CV_64F, kernel_ref)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(edge_map, vmin=-1, vmax=1), plt.axis('off'), plt.title('Your Second Derivative')
_ = plt.subplot(2, 2, 2), plt.imshow(np.abs(edge_map), vmin=0, vmax=1), plt.axis('off'), plt.title('Magnitude of Your Second Derivative')
_ = plt.subplot(2, 2, 3), plt.imshow(edge_map_ref, vmin=-1, vmax=1), plt.axis('off'), plt.title('Reference Second Derivative')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original image')

#### Section 2.2.2 Laplacian Kernel
A Laplace operator is defined as the sum of second derivatives of a function, with respect to the spatial variables. As such, for an image, the Laplacian would be as follows:

$ \nabla^2 I = \Large \frac{\partial^2 I}{\partial x^2} + \frac{\partial^2 I}{\partial x^2} $

Using this equation, a Laplacian kernel $L$ can be constructed as shown below.

$
L=
\begin{bmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0
\end{bmatrix}
$

And a wider family of Laplacian kernels $L_\alpha$ can be constructed, which might have different sensitivity towards diagonal edges.

$
L_\alpha=
\begin{bmatrix}
\alpha & 1-\alpha & \alpha \\
1-\alpha & -4 & 1-\alpha \\
\alpha & 1-\alpha & \alpha
\end{bmatrix}
$

Write a constructor function for a Laplacian kernel and see its effect on the sample image. Compare the results with what you saw with the second derivative kernels.

In [ ]:
# Implementing a Laplacian kernel constructor
def laplacianKernel(alpha : float) -> np.ndarray:
    """
    Parameters:
    - alpha : float [0 1]
        The alpha parameter in the Laplacian kernel.
    Returns:
    - kernel : np.ndarray
        A Laplacian kernel in np.ndarray format, with the specified direction and
        with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()


# Creating the kernels and applying the filter
alpha = 0
kernel = laplacianKernel(alpha)
kernel_ref = laplacianKernelRef(alpha)
edge_map = cv.filter2D(image, cv.CV_64F, kernel)
edge_map_ref = cv.filter2D(image, cv.CV_64F, kernel_ref)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(edge_map, vmin=-1, vmax=1), plt.axis('off'), plt.title('Your Laplacian')
_ = plt.subplot(2, 2, 2), plt.imshow(np.abs(edge_map), vmin=0, vmax=1), plt.axis('off'), plt.title('Magnitude of Your Laplacian')
_ = plt.subplot(2, 2, 3), plt.imshow(edge_map_ref, vmin=-1, vmax=1), plt.axis('off'), plt.title('Reference Laplacian')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original image')

**Note:** Filters such as Gaussian and mean, which preserve the global structure of the image while removing the smaller details are called *low-pass* filters. In contrast, filters like the derivatives which preserve the small-scale differences, but remove the large-scale features are called *low-pass* filters. You will learn more about such filters in the "Frequency Filtering" notebook.

## Section 3. Sharpening Filters
As you saw above, filters can be used to extract low-frequency (coarse) or high-frequency (fine) details from an image. Using these extracted details, we can boost the fine details to create a sharper image.

### Section 3.1. Sharpening with a Low-Pass filter
One approach to image sharpening, is by using the difference between an image and its blurred version (either from an average or a Gaussian filter). Since the low-pass filter removes the fine features, its difference with the original image should be the high-frequency details of the image. Therefore, with an image $I$ and a low-pass filter $\mathfrak{L}$, the sharpened image can be constructed using the following formula, where $c$ is a sharpening factor.

$
I_{sharp} = I + c \cdot (I - \mathfrak{L}[I]) = (1 + c) \cdot I - c \cdot \mathfrak{L}[I]
$

Use one of the low-pass filters that you created in the previous sections to sharpen a blurred image with this method.

In [ ]:
# Creating a blurred image
image_blurred = cv.GaussianBlur(image, (7, 7), 2)

# Setting the parameters
c = 4

# Creating the necessary objects for visualization
image_lp = None # The image passed through a low-pass filter
image_sharpened_lp = None # The sharpened image

# ====== YOUR CODE ======
raise NotImplementedError()


image_sharpened_lp_ref = sharpeningLPRef(image_blurred, c)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(image_sharpened_lp, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Sharpened Image')
_ = plt.subplot(2, 2, 2), plt.imshow(image_sharpened_lp_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Sharpened Image')
_ = plt.subplot(2, 2, 3), plt.imshow(image_blurred, vmin=0, vmax=1), plt.axis('off'), plt.title('Blurred Image')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

### Section 3.2. Unsharp Filtering
Another approach to image sharpening is by subtracting the Laplacian of an image from itself. This results in a new kernel which is called the *unsharp* kernel.

$
K_{unsharp} =
\begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0
\end{bmatrix}
$

A demonstration of how subtracting the second derivative of a function from itself would result in a sharper function can be seen below.

![image](figures/unsharp.jpg)

Needless to say, any type of Laplacian kernel can be used to create an unsharp kernel, and any factor of the Laplacian can be subtracted from the original image, so that different levels of sharpening are achieved.

Use a Laplacian kernel of your choice to sharpen an image via unsharp filtering.

In [ ]:
# Setting the parameters
c = 4

# Creating the necessary objects for visualization
image_sharpened_unsharp = None # The sharpened image

# ====== YOUR CODE ======
raise NotImplementedError()


image_sharpened_unsharp_ref = sharpeningUnsharpRef(image_blurred, c)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(image_sharpened_unsharp, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Sharpened Image')
_ = plt.subplot(2, 2, 2), plt.imshow(image_sharpened_unsharp_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Sharpened Image')
_ = plt.subplot(2, 2, 3), plt.imshow(image_blurred, vmin=0, vmax=1), plt.axis('off'), plt.title('Blurred Image')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

**Note:** You can see more variants of spatial filtering (harmonic, median, etc.) in the "Noise Removal" notebook.

# Scratchpad
You can use this section to try out different codes, without making a mess in the notebook. :)

In [ ]:
pass